In [1]:
from census import Census
from config import (api_key, gkey)
import pandas as pd
import requests
import json

In [8]:
url = "https://api.census.gov/data/2010/dec/sf1?get=PCT020B027,PCT020D027,PCT020E027,PCT021093,PCT021061,PCT021029,PCT021190,PCT021126,PCT021158,PCT020H027,PCT020A027,PCT020I027,PCT020C027,PCT020027,PCT020F027,PCT020G027,NAME&for=county:*&key=bdb18cf93fde53cedda10d62ee24bc6d29e147e5"
response = requests.get(url)
response_json = response.json()

census_pd = pd.DataFrame(response_json)

census_pd.columns = census_pd.iloc[0]
census_pd = census_pd[1:]

census_pd["County"] = census_pd["NAME"].str.split(pat=",", expand=True)[0]
census_pd["State"] = census_pd["NAME"].str.split(pat=",", expand=True)[1]

del census_pd["state"]
del census_pd["county"]
del census_pd["NAME"]

cols = census_pd.columns.drop(["County","State"])

census_pd[cols] = census_pd[cols].apply(pd.to_numeric)

census_pd["Homeless Pop"] = census_pd.sum(axis=1)

final_data = census_pd[["County", "State", "Homeless Pop"]]

In [9]:
final_data = final_data[final_data["Homeless Pop"] > 0]
final_data.head(20)

,County,State,Homeless Pop
14,Coffee County,Alabama,64
21,Dale County,Alabama,10
22,Dallas County,Alabama,102
26,Etowah County,Alabama,52
27,Fayette County,Alabama,40
31,Lee County,Alabama,103
33,Houston County,Alabama,103
35,Jefferson County,Alabama,1958
37,Lauderdale County,Alabama,79
43,Madison County,Alabama,957
